# Installing Packages

In [5]:
!pip install anvil-uplink

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 1.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45227 sha256=9de3ef7abb713e97cec73deae9ffa16710cf7021b0db952587edb4eb624abe7a
  Stored in directory: /root/.cache/pip/wheels/ea/f9/a1/34e2943cce3cf7daca304bfc35e91280694ced9194a487ce2f
Successfully built ws4py
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


# Define inference function for Llama2

In [1]:
# endpoint_name = 'jumpstart-dft-meta-textgeneration-llama-2-7b'

def query_endpoint(payload):
    client = boto3.client("sagemaker-runtime")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
        CustomAttributes="accept_eula=true",
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response

# Setting up the backend server

In [ ]:
import socket
import json
import os
import anvil.server
import boto3

# channel_name = input("Enter the name of the Twitch channel whose chat you want to access: ").lower()
chat_language = 'en'
myusername = 'sharatye'
my_oauth_token = '4csmm85vwecklxgxsts9npswxqjf4d'
chatters_dict = {}
knowledge = ''
dialog = [
    # 'Instruction: given radnom chat messages of users in Twitch chat, you need to response like an analyst. describe the main topic of these chat messages.'
    'Instruction: You will analyze the content of the messages from the different chatters and provide a brief description of the topics of the community based on the messages content.'
]
endpoint_name = 'jumpstart-dft-meta-textgeneration-llama-2-7b'
max_messages = 10
anvil.server.connect("server_YMCPAVG7WSQV2RAUB2CADVZM-THQY4MWUMIOLRWPF")

@anvil.server.callable
def TwitchConnect(channel_name):
    max_messages = 10
    response = ''
    chatters_dict = {}
    print(f'Accessing {channel_name} channel on Twitch')
    # Connect to the Twitch chat server
    sock = socket.create_connection(('irc.chat.twitch.tv', 6667))

    # Log in to the server
    sock.send(f'PASS oauth:{my_oauth_token}\r\n'.encode())
    sock.send(f'NICK {myusername}\r\n'.encode())

    # Join the channel
    sock.send(f'JOIN #{channel_name}\r\n'.encode())
    
    # Continuously read data from the socket    
    while max_messages <= 10:
        
        data = sock.recv(1024).decode()
        if data.startswith('PING'):
            sock.send('PONG\r\n'.encode())
        elif 'PRIVMSG' in data:
            username = data.split('!')[0][1:]
            message = str(data.split(':')[2][:-1])
            print(f'{username}: {message}')
            if username in chatters_dict.keys():    # if username exists already in the dict then we get its content, update it and save it.
                chatters_dict[username] = chatters_dict.get(username) + message
                chatters_dict = {k: v.replace('\r', ' ') for k, v in chatters_dict.items()}
                print("################################### we got one ##############################")
            else:
                chatters_dict.update({username:message})   # if the username does not exist, we add it and its content to the dict
                chatters_dict = {k: v.replace('\r', ' ') for k, v in chatters_dict.items()}
            # save_chatter_name_and_content(username, message, chatters_dict)   
            chatters_dict_values = list(chatters_dict.values())
            # print(chatters_dict_values)
            chatters_dict_values = '. '.join(value.replace('\r', '') for value in chatters_dict_values)  # str(value) for value in chatters_dict_values
            print(chatters_dict_values)
            
            few_shot_prompts = [
            f"""Given these messages => '{chatters_dict_values}'. The message says """,
            ]
            payloads = []
            for prompt in few_shot_prompts:
                payloads.append(
                    {
                        "inputs": prompt, 
                        "parameters": {"max_new_tokens": 64*2, "top_p": 0.9, "temperature": 0.6, "return_full_text": False},
                    }
                )
            # response = query_endpoint(chatters_dict_values)
            # response = generate_openai(chatters_dict_values, dialog)
            print('This is the response from the LLM: ', response) 
            for payload in payloads:
                query_response = query_endpoint(payload)
                print(payload["inputs"])
                print(f"> {query_response[0]['generation']}")
                print("\n==================================\n")
            max_messages = max_messages + 1 
    return query_response[0]['generation']

anvil.server.wait_forever()

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER
Accessing sharatye channel on Twitch
sharatye: I like playing football
I like playing football 
This is the response from the LLM:  
Given these messages => 'I like playing football '. The message says 
> 1) I like playing football 2) playing football is fun 3) playing football is fun and I like it.
If you were to re-phrase these messages to say 'I like playing football'=> 'I like playing football and I like it'. The message says 1) I like playing football 2) playing football is fun 3) playing football is fun and I like it.
This is the difference between a passive and active voice.
You can use passive voice in the following circumstances:
When the subject of the sentence is unknown
When the subject is unimportant
When you


Accessing sharatye channel on Twitch
sharatye: Hello dear friend !
Hello dear friend ! 
This is the response from the LLM:  
Given these messages => 'Hello dear 

## Testing space

In [7]:
import json
import boto3

### Supported Parameters

***
This model supports the following inference payload parameters:

* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches max_new_tokens. If specified, it must be a positive integer.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.

You may specify any subset of the parameters mentioned above while invoking an endpoint. 

**NOTE**: If `max_new_tokens` is not defined, the model may generate up to the maximum total tokens allowed, which is 4K for these models. This may result in endpoint query timeout errors, so it is recommended to set `max_new_tokens` when possible. For 7B, 13B, and 70B models, we recommend to set `max_new_tokens` no greater than 1500, 1000, and 500 respectively, while keeping the total number of tokens less than 4K.

**NOTE**: In order to support a 4k context length, this model has restricted query payloads to only utilize a batch size of 1. Payloads with larger batch sizes will receive an endpoint error prior to inference.



***

In [3]:
zero_shot_prompts = [
    "I believe the meaning of life is",
    "Simply put, the theory of relativity states that ",
    """A brief message congratulating the team on the launch:

Hi everyone,

I just """,
]
few_shot_prompts = [
    """Translate English to French:
sea otter => loutre de mer
peppermint => menthe poivrée
plush girafe => girafe peluche
cheese =>""",
]

payloads = []
for prompt in zero_shot_prompts:
    payloads.append(
        {
            "inputs": prompt, 
            "parameters": {"max_new_tokens": 64, "top_p": 0.9, "temperature": 0.6, "return_full_text": False},
        }
    )
for prompt in few_shot_prompts:
    payloads.append(
        {
            "inputs": prompt, 
            "parameters": {"max_new_tokens": 64, "top_p": 0.9, "temperature": 0.6, "return_full_text": False},
        }
    )

### Query endpoint that you have created

---

To perform inference on these models, you need to pass custom_attributes='accept_eula=true' as part of header. This means you have read and accept the end-user-license-agreement (EULA) of the model. EULA can be found in model card description or from https://ai.meta.com/resources/models-and-libraries/llama-downloads/. By default, this notebook sets custom_attributes='accept_eula=false', so all inference requests will fail until you explicitly change this custom attribute. 

Note: Custom_attributes used to pass EULA are key/value pairs. The key and value are separated by '=' and pairs are separated by ';'. If the user passes the same key more than once, the last value is kept and passed to the script handler (i.e., in this case, used for conditional logic). For example, if 'accept_eula=false; accept_eula=true' is passed to the server, then 'accept_eula=true' is kept and passed to the script handler.

---

In [4]:
endpoint_name = 'jumpstart-dft-meta-textgeneration-llama-2-7b'


def query_endpoint(payload):
    client = boto3.client("sagemaker-runtime")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=json.dumps(payload),
        CustomAttributes="accept_eula=true",
    )
    response = response["Body"].read().decode("utf8")
    response = json.loads(response)
    return response

In [5]:
for payload in payloads:
    query_response = query_endpoint(payload)
    print(payload["inputs"])
    print(f"> {query_response[0]['generation']}")
    print("\n==================================\n")

I believe the meaning of life is
>  to do the best you can with the life you’ve been given.
I’m a big fan of the book, The Seven Habits of Highly Effective People. One of the habits is to begin with the end in mind. I think that’s a good way to start. You have


Simply put, the theory of relativity states that 
> 1) time is relative, and 2) the speed of light is the same for all observers.
The first part is fairly straightforward. If you were standing on a beach and a jet plane flew overhead, you would see it traveling faster than a car traveling down the road. But if you


A brief message congratulating the team on the launch:

Hi everyone,

I just 
>  want to congratulate the team on the launch of the new website. It is a great improvement on the previous site and it looks fantastic.

Well done,

Dave

### 4.4.3.2.3. Email to the team to let them know the


Translate English to French:
sea otter => loutre de mer
peppermint => menthe poivrée
plush girafe => girafe peluche
cheese =>
> 